In [21]:
import sys
import struct
import math
import time
import random
import mysql.connector 
from matplotlib import pyplot as plot
from mysql.connector import errorcode
import numpy as np

# Criação de listas
lista_decibel = []
lista_ambiente = []
lista_espaco = []
lista_tempo = []

# Configurações da gravação de áudio simulado
CHUNK = 1024
RATE = 44100

# Define o limiar de ruído máximo permitido (em dB)
MAX_NOISE_LEVEL = 45

def calculate_noise_level(data):
    # Converte os dados do áudio em um array de números inteiros
    data_int = struct.unpack(str(len(data)) + 'B', data)

    # Calcula a energia do sinal
    rms = math.sqrt(sum([(x - 128) ** 2 for x in data_int]) / len(data_int))

    # Calcula o nível de ruído em dB usando a escala A de ponderação de frequência
    decibel = 20 * math.log10(rms)

    return decibel

def medir_nivel_ruido(tempo_medicao):
    inicio_processamento = time.time()

    # Inicializa a variável como verdadeira
    ambiente_adequado = True
    ambiente_simulado = True

    #Variável que vai pro banco
    ambiente = ""
    ambiente_2 = ""

    # Calcula o número de chunks para gerar com base no tempo de medição
    num_chunks = int(RATE / CHUNK * tempo_medicao)

    # Inicializa a lista para armazenar os dados de áudio simulados
    data_list = []

    # Gera os dados de áudio simulados
    for i in range(num_chunks):
        data = np.random.randint(-32768, 32767, CHUNK, dtype=np.int16).tobytes()
        data_list.append(data)

    # Concatena os dados de áudio simulados em uma única string
    data_concatenado = b''.join(data_list)

    # Calcula o nível de ruído em dB
    decibel = calculate_noise_level(data_concatenado)
    decibel_simulado = round(random.normalvariate(decibel, 40), 2)
    while (decibel_simulado <= 0):
        decibel_simulado = round(random.normalvariate(decibel, 40), 2)

    # Imprime as informações do nível de ruído
    print(f"Nível de ruído durante {tempo_medicao} segundos: {decibel:.2f} dB")

    # Verifica se o nível de ruído está acima do limiar máximo permitido
    if decibel > MAX_NOISE_LEVEL:
        print("O ambiente não está adequado para o sono do paciente")
        ambiente_adequado = False
        ambiente = "Não Adequado"
    else:
        print("O ambiente está adequado para o sono do paciente")
        ambiente_adequado = True
        ambiente = "Adequado"

    # Imprime as informações do nível de ruído
    print(f"Nível de ruído simulado durante {tempo_medicao} segundos: {decibel_simulado:.2f} dB")

    # Verifica se o nível de ruído está acima do limiar máximo permitido
    if decibel_simulado > MAX_NOISE_LEVEL:
        print("O ambiente simulado não está adequado para o sono do paciente")
        ambiente_simulado = False
        ambiente_2 = "Não Adequado"
    else:
        print("O ambiente simulado está adequado para o sono do paciente")
        ambiente_simulado = True
        ambiente_2 = "Adequado"

    # Simulação da gravação do áudio
    time.sleep(0.01)  # Simula o tempo de gravação do áudio

    lista_decibel.append(decibel_simulado)
    lista_ambiente.append(ambiente_simulado)

    fim_processamento = time.time()
    duracao = fim_processamento - inicio_processamento
    espaco = sys.getsizeof(lista_decibel) / (1024 * 1024)

    lista_espaco.append(espaco)
    lista_tempo.append(duracao)

    return ambiente_adequado, ambiente_simulado

num_amostras = 1000
tempo_medicao = 1
lista_ambiente_adequado = []
lista_ambiente_simulado = []

for i in range(num_amostras):
    ambiente_adequado, ambiente_simulado = medir_nivel_ruido(tempo_medicao)
    lista_ambiente_adequado.append(ambiente_adequado)
    lista_ambiente_simulado.append(ambiente_simulado)


Nível de ruído durante 1 segundos: 37.37 dB
O ambiente está adequado para o sono do paciente
Nível de ruído simulado durante 1 segundos: 3.60 dB
O ambiente simulado está adequado para o sono do paciente
Nível de ruído durante 1 segundos: 37.39 dB
O ambiente está adequado para o sono do paciente
Nível de ruído simulado durante 1 segundos: 93.08 dB
O ambiente simulado não está adequado para o sono do paciente
Nível de ruído durante 1 segundos: 37.34 dB
O ambiente está adequado para o sono do paciente
Nível de ruído simulado durante 1 segundos: 57.51 dB
O ambiente simulado não está adequado para o sono do paciente
Nível de ruído durante 1 segundos: 37.40 dB
O ambiente está adequado para o sono do paciente
Nível de ruído simulado durante 1 segundos: 72.96 dB
O ambiente simulado não está adequado para o sono do paciente
Nível de ruído durante 1 segundos: 37.39 dB
O ambiente está adequado para o sono do paciente
Nível de ruído simulado durante 1 segundos: 4.58 dB
O ambiente simulado está ade

KeyboardInterrupt: 